In [1]:
%load_ext autoreload
%autoreload 2
import os
import glob
import h5py
import numpy as np
from IPython.display import display
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import wandb

from dl_utils.utils.dataset import split_train_valid, viz_dataloader, hdf5_dataset
from dl_utils.utils.utils import list_to_dict, find_last_epoch_file, sort_tasks_by_size

from dl_utils.training.build_model import resnet50_, fpn_resnet50_classification, xcit_small
from dl_utils.training.train_functions import train_epochs
from dl_utils.analysis.confusion_matrix import confusion_matrix, plot_cm
from dl_utils.packed_functions import generate_confusion_matrix_batch_full_val_set

/mnt/scratch/home/yichen/anaconda3/envs/symmetry/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ResNet 50

In [2]:
task_orders = ['10m', '1k', '5k', '10k', '50k', '100k', '500k', '1m', '2m', '5m', ]
ds_size_list = [10000000, 1000, 5000, 10000, 50000, 100000, 500000, 1000000, 2000000, 5000000]
model_path_list = sort_tasks_by_size(glob.glob('../../../models/ResNet50/*'), task_orders)
print(model_path_list)

ds_path_info = {'imagenet': '../../../datasets/imagenet_v5_rot_10m_fix_vector.h5',
                'noise': '../../../datasets/noise_v5_rot_1m_fix_vector.h5',
                'atom': '../../../datasets/atom_v5_rot_1m_fix_vector.h5'}
running_specs = {'batch_size': 3500, 
                'num_workers': 12, 
                'save_path': '../../../results/ResNet50/',
                'device_ids': [5]}

['../../../models/ResNet50/03132025-ResNet50-benchmark-10m', '../../../models/ResNet50/03132025-ResNet50-benchmark-1k', '../../../models/ResNet50/03132025-ResNet50-benchmark-5k', '../../../models/ResNet50/03132025-ResNet50-benchmark-10k', '../../../models/ResNet50/03132025-ResNet50-benchmark-50k', '../../../models/ResNet50/03132025-ResNet50-benchmark-100k', '../../../models/ResNet50/03132025-ResNet50-benchmark-500k', '../../../models/ResNet50/03132025-ResNet50-benchmark-1m', '../../../models/ResNet50/03132025-ResNet50-benchmark-2m', '../../../models/ResNet50/03132025-ResNet50-benchmark-5m']


### only do the cross-validation on the full set

In [3]:
model = resnet50_(3, 17)
stats = generate_confusion_matrix_batch_full_val_set(model, model_path_list, ds_size_list, ds_path_info, running_specs)

Running on cuda:5
Loading model from ../../../models/ResNet50/03132025-ResNet50-benchmark-10m/epoch_23.pth
Confusion Matrix for 03132025-ResNet50-benchmark-10m:


100%|█████████████████████████████████████████████████████████████████████████████████| 287/287 [13:41<00:00,  2.86s/it]

Sum for true labels:


,p1,p2,pm,pg,cm,pmm,pmg,pgg,cmm,p4,p4m,p4g,p3,p3m1,p31m,p6,p6m
0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0


Accuracy for these batches: 88.38%


...

Accuracy for these batches: 86.92%


100%|█████████████████████████████████████████████████████████████████████████████████| 287/287 [12:32<00:00,  2.62s/it]

Sum for true labels:


,p1,p2,pm,pg,cm,pmm,pmg,pgg,cmm,p4,p4m,p4g,p3,p3m1,p31m,p6,p6m
0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0,58900.0


Accuracy for these batches: 96.16%


In [4]:
stats

{'03132025-ResNet50-benchmark-10m': {'03132025-ResNet50-benchmark-10m-atom_cv': 0.8837681014680915,
  '03132025-ResNet50-benchmark-10m-noise_cv': 0.9619464695895336},
 '03132025-ResNet50-benchmark-1k': {'03132025-ResNet50-benchmark-1k-atom_cv': 0.10467791870568262,
  '03132025-ResNet50-benchmark-1k-noise_cv': 0.08680115849395785},
 '03132025-ResNet50-benchmark-5k': {'03132025-ResNet50-benchmark-5k-atom_cv': 0.3531469090182762,
  '03132025-ResNet50-benchmark-5k-noise_cv': 0.3523000099870169},
 '03132025-ResNet50-benchmark-10k': {'03132025-ResNet50-benchmark-10k-atom_cv': 0.40702786377708977,
  '03132025-ResNet50-benchmark-10k-noise_cv': 0.3758064516129032},
 '03132025-ResNet50-benchmark-50k': {'03132025-ResNet50-benchmark-50k-atom_cv': 0.38722161190452414,
  '03132025-ResNet50-benchmark-50k-noise_cv': 0.5916848097473285},
 '03132025-ResNet50-benchmark-100k': {'03132025-ResNet50-benchmark-100k-atom_cv': 0.43859083191850595,
  '03132025-ResNet50-benchmark-100k-noise_cv': 0.685559772296015

In [5]:
stats

{'03132025-ResNet50-benchmark-10m': {'03132025-ResNet50-benchmark-10m-atom_cv': 0.8837681014680915,
  '03132025-ResNet50-benchmark-10m-noise_cv': 0.9619464695895336},
 '03132025-ResNet50-benchmark-1k': {'03132025-ResNet50-benchmark-1k-atom_cv': 0.10467791870568262,
  '03132025-ResNet50-benchmark-1k-noise_cv': 0.08680115849395785},
 '03132025-ResNet50-benchmark-5k': {'03132025-ResNet50-benchmark-5k-atom_cv': 0.3531469090182762,
  '03132025-ResNet50-benchmark-5k-noise_cv': 0.3523000099870169},
 '03132025-ResNet50-benchmark-10k': {'03132025-ResNet50-benchmark-10k-atom_cv': 0.40702786377708977,
  '03132025-ResNet50-benchmark-10k-noise_cv': 0.3758064516129032},
 '03132025-ResNet50-benchmark-50k': {'03132025-ResNet50-benchmark-50k-atom_cv': 0.38722161190452414,
  '03132025-ResNet50-benchmark-50k-noise_cv': 0.5916848097473285},
 '03132025-ResNet50-benchmark-100k': {'03132025-ResNet50-benchmark-100k-atom_cv': 0.43859083191850595,
  '03132025-ResNet50-benchmark-100k-noise_cv': 0.685559772296015

In [6]:
import pandas as pd
df = pd.read_csv('../../../results/history/training_summary_latest.csv')
df

,Name,Model,Dataset_size,train_loss,valid_loss,train_accuracy,valid_accuracy,noise_accuracy,noise_loss,atom_accuracy,atom_loss
0,ResNet50-1k,ResNet50,1000,0.024114,8.707681,0.990000,0.228000,0.079000,9.710158,0.124000,9.607305
1,ResNet50-5k,ResNet50,5000,0.026530,3.826476,0.990600,0.552800,0.359200,4.911996,0.359400,7.174067
2,ResNet50-10k,ResNet50,10000,0.028572,3.448092,0.990300,0.598800,0.368700,4.815066,0.414900,6.321324
3,ResNet50-50k,ResNet50,50000,0.027217,1.041223,0.990240,0.869840,0.588840,3.003041,0.388060,7.584784
4,ResNet50-100k,ResNet50,100000,0.136364,1.518383,0.958889,0.633200,0.114340,4.077415,0.393780,3.548233
5,ResNet50-500k,ResNet50,500000,0.030453,0.143844,0.989220,0.975808,0.885926,0.899102,0.703082,3.702238
6,ResNet50-1m,ResNet50,1000000,0.030649,0.090277,0.989165,0.982200,0.938441,0.486521,0.745000,3.324254
7,ResNet50-2m,ResNet50,2000000,0.044772,0.053977,0.985556,0.982848,0.940066,0.264464,0.780175,1.657540
8,ResNet50-5m,ResNet50,5000000,0.011643,0.020937,0.997277,0.994947,0.935147,0.273390,0.842422,1.183674
9,ResNet50-10m,ResNet50,10000000,0.037016,0.036806,0.986667,0.986712,0.961947,0.134230,0.883770,0.979830


In [7]:
acc_update_dict = stats

# Helper to convert Dataset_size into suffix (1k, 5k, 1m, etc.)
def size_suffix(size):
    if size >= 1_000_000:
        return f"{int(size / 1_000_000)}m"
    elif size >= 1000:
        return f"{int(size / 1000)}k"
    else:
        return str(size)

# Update DataFrame
for i, row in df.iterrows():
    model_name = row['Model']
    size = size_suffix(row['Dataset_size'])
    dict_key = f"03132025-{model_name}-benchmark-{size}"
    
    if dict_key in acc_update_dict:
        atom_cv_key = f"{dict_key}-atom_cv"
        noise_cv_key = f"{dict_key}-noise_cv"
        df.at[i, 'atom_accuracy'] = acc_update_dict[dict_key].get(atom_cv_key, row['atom_accuracy'])
        df.at[i, 'noise_accuracy'] = acc_update_dict[dict_key].get(noise_cv_key, row['noise_accuracy'])

df.to_csv('../../../results/history/training_summary_latest_full_val_add.csv', index=False)


In [8]:
df

,Name,Model,Dataset_size,train_loss,valid_loss,train_accuracy,valid_accuracy,noise_accuracy,noise_loss,atom_accuracy,atom_loss
0,ResNet50-1k,ResNet50,1000,0.024114,8.707681,0.990000,0.228000,0.086801,9.710158,0.104678,9.607305
1,ResNet50-5k,ResNet50,5000,0.026530,3.826476,0.990600,0.552800,0.352300,4.911996,0.353147,7.174067
2,ResNet50-10k,ResNet50,10000,0.028572,3.448092,0.990300,0.598800,0.375806,4.815066,0.407028,6.321324
3,ResNet50-50k,ResNet50,50000,0.027217,1.041223,0.990240,0.869840,0.591685,3.003041,0.387222,7.584784
4,ResNet50-100k,ResNet50,100000,0.136364,1.518383,0.958889,0.633200,0.685560,4.077415,0.438591,3.548233
5,ResNet50-500k,ResNet50,500000,0.030453,0.143844,0.989220,0.975808,0.885399,0.899102,0.705415,3.702238
6,ResNet50-1m,ResNet50,1000000,0.030649,0.090277,0.989165,0.982200,0.938442,0.486521,0.745620,3.324254
7,ResNet50-2m,ResNet50,2000000,0.044772,0.053977,0.985556,0.982848,0.948342,0.264464,0.790942,1.657540
8,ResNet50-5m,ResNet50,5000000,0.011643,0.020937,0.997277,0.994947,0.961571,0.273390,0.869208,1.183674
9,ResNet50-10m,ResNet50,10000000,0.037016,0.036806,0.986667,0.986712,0.961946,0.134230,0.883768,0.979830


### 100k subset of the full set

In [3]:
model = resnet50_(3, 17)
stats = generate_confusion_matrix_batch_full_val_set(model, model_path_list, ds_size_list, ds_path_info, running_specs, subset_size=100000)

Running on cuda:5
Loading model from ../../../models/ResNet50/03132025-ResNet50-benchmark-10m/epoch_23.pth
Confusion Matrix for 03132025-ResNet50-benchmark-10m:


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [01:19<00:00,  2.75s/it]


Sum for true labels:


,p1,p2,pm,pg,cm,pmm,pmg,pgg,cmm,p4,p4m,p4g,p3,p3m1,p31m,p6,p6m
0,5922.0,5936.0,5747.0,5910.0,5832.0,5821.0,5751.0,5984.0,5896.0,5867.0,5846.0,5923.0,5909.0,5814.0,5952.0,5888.0,6002.0


Accuracy for these batches: 88.32%


...

Accuracy for these batches: 86.76%


100%|███████████████████████████████████████████████████████████████████████████████████| 29/29 [01:16<00:00,  2.64s/it]


Sum for true labels:


,p1,p2,pm,pg,cm,pmm,pmg,pgg,cmm,p4,p4m,p4g,p3,p3m1,p31m,p6,p6m
0,5940.0,5774.0,5955.0,5828.0,5860.0,5874.0,6006.0,5785.0,5869.0,6010.0,5823.0,5797.0,5896.0,5916.0,5945.0,5894.0,5828.0


Accuracy for these batches: 96.20%


In [4]:
import pandas as pd
df = pd.read_csv('../../../results/history/training_summary_latest.csv')
df

,Name,Model,Dataset_size,train_loss,valid_loss,train_accuracy,valid_accuracy,noise_accuracy,noise_loss,atom_accuracy,atom_loss
0,ResNet50-1k,ResNet50,1000,0.024114,8.707681,0.990000,0.228000,0.079000,9.710158,0.124000,9.607305
1,ResNet50-5k,ResNet50,5000,0.026530,3.826476,0.990600,0.552800,0.359200,4.911996,0.359400,7.174067
2,ResNet50-10k,ResNet50,10000,0.028572,3.448092,0.990300,0.598800,0.368700,4.815066,0.414900,6.321324
3,ResNet50-50k,ResNet50,50000,0.027217,1.041223,0.990240,0.869840,0.588840,3.003041,0.388060,7.584784
4,ResNet50-100k,ResNet50,100000,0.136364,1.518383,0.958889,0.633200,0.114340,4.077415,0.393780,3.548233
5,ResNet50-500k,ResNet50,500000,0.030453,0.143844,0.989220,0.975808,0.885926,0.899102,0.703082,3.702238
6,ResNet50-1m,ResNet50,1000000,0.030649,0.090277,0.989165,0.982200,0.938441,0.486521,0.745000,3.324254
7,ResNet50-2m,ResNet50,2000000,0.044772,0.053977,0.985556,0.982848,0.940066,0.264464,0.780175,1.657540
8,ResNet50-5m,ResNet50,5000000,0.011643,0.020937,0.997277,0.994947,0.935147,0.273390,0.842422,1.183674
9,ResNet50-10m,ResNet50,10000000,0.037016,0.036806,0.986667,0.986712,0.961947,0.134230,0.883770,0.979830


In [5]:
acc_update_dict = stats

# Helper to convert Dataset_size into suffix (1k, 5k, 1m, etc.)
def size_suffix(size):
    if size >= 1_000_000:
        return f"{int(size / 1_000_000)}m"
    elif size >= 1000:
        return f"{int(size / 1000)}k"
    else:
        return str(size)

# Update DataFrame
for i, row in df.iterrows():
    model_name = row['Model']
    size = size_suffix(row['Dataset_size'])
    dict_key = f"03132025-{model_name}-benchmark-{size}"
    
    if dict_key in acc_update_dict:
        atom_cv_key = f"{dict_key}-atom_cv"
        noise_cv_key = f"{dict_key}-noise_cv"
        df.at[i, 'atom_accuracy'] = acc_update_dict[dict_key].get(atom_cv_key, row['atom_accuracy'])
        df.at[i, 'noise_accuracy'] = acc_update_dict[dict_key].get(noise_cv_key, row['noise_accuracy'])

df.to_csv('../../../results/history/training_summary_latest_full_val_100k_subset.csv', index=False)
df

,Name,Model,Dataset_size,train_loss,valid_loss,train_accuracy,valid_accuracy,noise_accuracy,noise_loss,atom_accuracy,atom_loss
0,ResNet50-1k,ResNet50,1000,0.024114,8.707681,0.990000,0.228000,0.086520,9.710158,0.106310,9.607305
1,ResNet50-5k,ResNet50,5000,0.026530,3.826476,0.990600,0.552800,0.352330,4.911996,0.356290,7.174067
2,ResNet50-10k,ResNet50,10000,0.028572,3.448092,0.990300,0.598800,0.376620,4.815066,0.407660,6.321324
3,ResNet50-50k,ResNet50,50000,0.027217,1.041223,0.990240,0.869840,0.590300,3.003041,0.387540,7.584784
4,ResNet50-100k,ResNet50,100000,0.136364,1.518383,0.958889,0.633200,0.686660,4.077415,0.439280,3.548233
5,ResNet50-500k,ResNet50,500000,0.030453,0.143844,0.989220,0.975808,0.884770,0.899102,0.705720,3.702238
6,ResNet50-1m,ResNet50,1000000,0.030649,0.090277,0.989165,0.982200,0.938220,0.486521,0.746510,3.324254
7,ResNet50-2m,ResNet50,2000000,0.044772,0.053977,0.985556,0.982848,0.948980,0.264464,0.790490,1.657540
8,ResNet50-5m,ResNet50,5000000,0.011643,0.020937,0.997277,0.994947,0.961960,0.273390,0.867620,1.183674
9,ResNet50-10m,ResNet50,10000000,0.037016,0.036806,0.986667,0.986712,0.961820,0.134230,0.883190,0.979830
